<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex07notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex07notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 事前学習済みモデルの利用
----






近年のいわゆる人工知能（AI, Airtificial Intelligence）のシステムの裏には，ニューラルネットワークをはじめとする様々な機械学習の技術があります．難しい課題をこなせるようなシステムを作るためには，学習データを大量に用意して大規模な（パラメータ数が多く複雑な）モデルを学習させることが必要なわけですが，そのような学習をすべて自前でやろうとすると，多大な手間と時間がかかります．

このような手間と時間を軽減する方法として，画像認識，音声認識，自然言語処理等の汎用性の高い問題を解くような学習モデルを，大量のデータを用いて事前に学習させておき，得られたモデルを個別の問題ごとに微調整して使う，というやり方がよく用いられます．

----
### 1000種類の画像識別を学習済みのニューラルネットを動かしてみよう


例として，VGG16 というニューラルネットワークの事前学習済みモデルを動かしてみます．VGG16は，イギリスオックスフォード大学の [Visual Geometry Group(VGG)](https://www.robots.ox.ac.uk/~vgg/) という研究グループが作った VGG-net というニューラルネットワークモデルのうちの，層が16層あるものです．
VGG-net は，2014年に行われた [ILSVRC2014](https://www.image-net.org/challenges/LSVRC/2014/) という画像識別のコンペティションで世界第2位となったニューラルネットワークモデルです．
ILSVRC2014では，[ImageNet](https://www.image-net.org/) という大規模な画像データセットの中から選ばれた 1000 種類の物体の画像約 120 万枚を学習データとしていました．

参考文献: [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

---
#### 準備

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
import pickle
import torch
from torchvision import models, transforms
from PIL import Image
import json

##### 準備その1: 実験に使う画像データの準備

**注意: 以下の画像をこの実験以外の目的で使用してはいけません**

In [ ]:
!wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/animalphoto.pickle
with open('animalphoto.pickle', 'rb') as f:
    hoge = pickle.load(f)

imgList = []
for d in hoge:
    img = Image.frombytes(**d)
    imgList.append(img)
    fig, ax = plt.subplots(1)
    ax.imshow(img)
    ax.axis('off')
    plt.show()

##### 準備その2: クラスの番号とクラスラベルの対応表の作成

In [ ]:
# ImageNet クラスラベルを表すJSONファイルを入手
!wget -nc https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
class_index = json.load(open('imagenet_class_index.json', 'r'))

K = len(class_index)
labels = {}
for ik, key in enumerate(class_index.keys()):
    labels[ik] = f'{class_index[key][0]} {class_index[key][1]}'
    if 276 <= ik < 300: 
        print(ik, labels[ik])

上記の出力は，1000クラスのうちの一部のものの名前を表します．「ネコ」みたいなものも1クラスではなく，281番 'tabby'（ぶち柄の猫），282番 'tiger_cat'（トラ縞の猫）等々のように分かれています．

##### 準備3: 事前学習済みニューラルネットのパラメータの入手

規模の大きいネットワークでパラメータがたくさんありますので，読み込みに少し時間がかかります．

In [ ]:
weights = models.VGG16_Weights.IMAGENET1K_V1
preprocess = weights.transforms()
model = models.vgg16(weights=weights)
model.eval()
print(model)

上記のセルを実行すると，次のことが行われます．
1. VGG16のネットワークモデルを作成する．
1. ImageNetで学習済みのパラメータの値を読み込んでこのネットワークのパラメータに設定する．
1. ネットワークモデルの構造を表示する．

画像を扱うニューラルネットでは，「畳み込み層」(convolutional layer)という特別な構造がよく用いられます．
VGG16は，畳み込み層が13層（上記の出力の `Conv2d` というのが畳み込み層），全結合層（`Linear`，階層型ニューラルネットワークとして説明したものはこれ）3層の計16層から成っています（これ以外にもいくつか特殊な層が間に挟まってます）．

---
#### 実験1

上記のサンプル画像たちを VGG16 で識別させてみましょう．


In [ ]:
#@title サンプル画像の識別
#@markdown 以下で 0 から 3 までの整数を選ぶと4つのサンプル画像の中から一つを選ぶことができます．
i = 0 #@param [0, 1, 2, 3] {type: 'raw'}
img = imgList[i]

# 画像を表示
fig, ax = plt.subplots(1)
ax.imshow(img)
ax.axis('off')
plt.show()

# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(imgList[i]), axis=0)
Y = model(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')

上記の画像の下の出力は，1000個のクラスの中で，ネットワークの出力の値が大きかったもの上位5位までの，出力の値とクラス名を表します．
ネットワークの出力層の活性化関数は softmax ですので，出力の値は0から1でかつ1000子の出力の値すべての和が 1 となっています（一般のロジスティック回帰モデルの説明参照）．
これらの値は，そのクラスと判定することの「確信度」と解釈できます．

---
#### 実験2

自分で用意した画像でも実験してみましょう．



(1) まずは，ウェブ等で適当な画像を探して手元に保存しましょう．JPEG（拡張子は `.jpg` や `.jpeg` 等）やPNG（`.png`）等の形式のものが扱えます．ファイル名が長かったり日本語を含んでいる場合は，短い名前に変更しておくのがよいです．

(2) 以下のセルを実行して，ファイルをアップロードします．

In [ ]:
# Colab へファイルをアップロード
from google.colab import files
rv = files.upload()

# ファイル一覧
! ls

(3) 以下のセルの1行目のファイル名を上記でアップロードしたものに変えて実行しましょう．アルファベット大文字小文字の区別もありますので注意．うまくいけば画像が表示されるはずです．

In [ ]:
fn = 'hoge.jpg' # ファイル名を変更しましょう

myimg = Image.open(fn)
if myimg.mode != 'RGB':
    myimg = myimg.convert('RGB')
fig, ax = plt.subplots(1)
ax.imshow(myimg)
ax.axis('off')
plt.show()

(4) 以下のセルを実行すれば結果が表示されます．

In [ ]:
# VGG16 ネットワークに入力して出力を得る
X = torch.unsqueeze(preprocess(myimg), axis=0)
Y = model(X)
Z = torch.nn.functional.softmax(Y, dim=1)
P = Z[0].detach().numpy()

# 出力の値の大きかった方から5つを表示
for i, ik in enumerate(np.argsort(-P)[:5]):
    print(f'rank{i+1}: {P[ik]:.6f} {labels[ik]}')


実験1は動物の例ばかりでしたが，1000のクラスの中にはそれ以外にも様々なものがあります（「人間」を表すクラスはありません）．いろいろ試してみてください．面白い／不思議な結果が得られたら takataka に見せてくれると喜びます．


---
### ファインチューニング

notebook の冒頭でも説明したように，機械学習モデルを一から学習させるには手間と時間がかかります．そのため，自分が扱うタスク（課題）でモデルを一から学習させるかわりに，類似のタスクで学習済みのモデル（事前学習済みモデル）を入手して，そのモデルを自分のタスク向けに微調整して使うことがよくあります．この微調整のことを **ファインチューニング**(fine-tuning) といいます．


事前学習済みのニューラルネットをファインチューニングするには，典型的には次のような手順をとります（下図参照）．

1. 自分のタスクと類似したタスクで学習済みの事前学習済みモデルを入手する．↑の VGG16 は，画像認識をやりたい場合に使える事前学習済みモデルの一例です．
1. 入手したモデルの出力側の一部を自分のタスクに合わせて新規に作り直し，自分の用意したデータでモデルを学習させる．このとき，パラメータを修正するのは新規に用意した部分のみとし，入力側のパラメータは事前学習済みモデルのものに固定しておく（注）．

※注: 事前学習モデルのパラメータを初期値として，入力側のパラメータも含めてモデル全体を学習させることもよくあります．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/transferlearning.png">

このような方法を採ることには，次のようなメリットがあります．

1. 大きなニューラルネットを一から学習させずに済むので，学習にかける手間と時間を軽減できる．
1. 少ないデータで大きなニューラルネットを一から学習させると過適合の心配があるが，このような方法を採ることで過適合を避けて識別率等の性能の高いモデルを得やすい．
1. 大規模データで学習した事前学習済みモデルは，その入力側の部分が特徴抽出の仕組みとして汎化性能の高い優れたものになっていることが多い．それを流用できるので，ファインチューニングしたモデルも高い性能を期待できる．

例えば，VGG16 のような事前学習済みモデルをファインチューニングすることで，新たな画像認識タスクを解く高性能なニューラルネットを手軽に作ることができます．